In [12]:
# import required module
import os
import csv
import ast
import re

In [13]:
directory="5ch/"

In [14]:
def category(directory):
    # iterate over files in
    # that directory
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file

        if os.path.isfile(f) and filename.endswith('.csv'):
            with open(f,'r',encoding='utf-8') as file:
                reader=csv.DictReader(file)
                fieldnames=reader.fieldnames
                rows=list(reader)
            yield fieldnames,rows

In [15]:
def filterByWords(usercontent):
    if len(usercontent['content'])>20:
        return usercontent
    else:
        return None
    
def filterByKeyword(usercontent):
    pattern=r'.*(経験談|失敗談).*'
    m=re.match(pattern=pattern,string=usercontent['content'])
    if m:
        return usercontent
    else:
        return None
    
def filterMain(usercontent):
    usercontent['content'].replace('\u3000',' ')
    return filterByWords(usercontent),filterByKeyword(usercontent)

In [16]:
def extractPost(posts):
    for post in posts:
        str_content=post['content']
        dict_content=ast.literal_eval(str_content)
        yield post,dict_content

In [17]:
def extractUsercontent(content):
    for i in range(len(content)):
        usercontent=content[i+1]
        yield i+1,usercontent

In [18]:
def ExportToCategoryFile(directory,filename,fieldnames,posts):
    with open(directory+filename,'w',encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(posts)

In [19]:
def main():
    categorylist=category(directory)
    for fieldnames,posts in categorylist:
        posts_updated1=[]
        posts_updated2=[]
        for post,dict_content in extractPost(posts):
            usercontents_updated1,usercontents_updated2={},{}
            for user_no, usercontent in extractUsercontent(dict_content):
                    filter1,filter2=filterMain(usercontent)
                    if filter1:
                        usercontents_updated1.update({user_no:filter1})
                        if filter2:
                             usercontents_updated2.update({user_no:filter2})
                             
            if usercontents_updated1:
                post_updated1=post.copy()
                post_updated1.update({'content':str(usercontents_updated1)})
                posts_updated1.append(post_updated1)
                if usercontents_updated2:
                    post_updated2=post.copy()
                    post_updated2.update({'content':str(usercontents_updated2)})
                    posts_updated2.append(post_updated2)
        if posts_updated1:
            filename='-'.join([post['category'],post['tag']])+'.csv'
            ExportToCategoryFile('5ch_wordlimit/',filename,fieldnames,posts_updated1)
            if posts_updated2:
                ExportToCategoryFile('5ch_keyword/',filename,fieldnames,posts_updated2) 
                

In [24]:
import sys
csv.field_size_limit(int(sys.maxsize))
main()

In [27]:
directory1="5ch_keyword/"
directory2="5ch_wordlimit/"
def checkRowNum(directory):
    categorylist=category(directory)
    counter=0
    for fieldnames,posts in categorylist:
        counter+=len(posts)
    return counter

print(f'keyword: {checkRowNum(directory1)} wordlimit:{checkRowNum(directory2)}')


keyword: 128 wordlimit:37394
